In [ ]:
import yfinance as yf
import numpy as np
from itertools import product
import json
import random 
import numpy as np
import pandas as pd
from pandas_datareader.data import DataReader
from dimod import Integer, Binary
from dimod import quicksum 
from dimod import ConstrainedQuadraticModel
import yfinance as yf
from dwave.system import LeapHybridDQMSampler, LeapHybridCQMSampler 
import matplotlib.pyplot as plt

In [ ]:
Data = yf.download(['MSFT','META','AAPL'], start ='2022-01-01')['Adj Close']
assets = Data.columns.tolist()
print(Data.head())
returns = Data.pct_change(1).dropna()
cov = np.array(returns.cov())
mean = returns.mean()
prices = Data.iloc[-1]
print(cov.shape)
print(cov)
print(mean.shape)
print(mean.head())
print("below are the prices")
print(prices.shape)
print(prices)

In [ ]:
(Data/Data.iloc[0]).plot()

In [ ]:
iv = [Integer(f'x_{i}', upper_bound=10) for i in range(3)]
print(iv)
cqm = ConstrainedQuadraticModel()
cqm.set_objective(quicksum(quicksum(iv[i]*iv[j]*prices[i]*prices[j]*cov[i,j] for j in range(3)) for i in range(3)))
budget = 10000
cqm.add_constraint(quicksum(prices[i]*iv[i] for i in range(3)) <=budget)
cqm.add_constraint(quicksum(prices[i]*iv[i] for i in range(3))  >= 0.98 * budget)

In [ ]:
cqm_sampler = LeapHybridCQMSampler(token='DEV-9756b9b04acfa6a267f828b5ac3cac123338fa1b')
sample_set = cqm_sampler.sample_cqm(cqm,label='investment optimization')
print (sample_set)

In [ ]:
a = sample_set.aggregate().record
cols  = list(a.dtype.names)
b = []

for i in range(a.shape[0]):
    b.append(list (a[i]))
    result = pd.DataFrame(data=b,columns=cols)
    result = result.sort_values('energy', ascending= False)

    
#index_names = result[result['is_feasible'] == False ].index
#result.drop(index_names,inplace=True)
ans = result['sample'].iloc[-1]

optim_weights = ans 
number_stocks = np.array(list(optim_weights))
number_stocks = np.reshape(number_stocks,(number_stocks.shape[0],))
print(number_stocks)
prices = np.reshape(prices,(prices.shape[0],))
optim_weights = np.multiply(number_stocks,prices)
optim_weights = optim_weights / np.sum(optim_weights)
print(optim_weights)

In [ ]:
plt.pie(list(np.reshape(optim_weights,(-1,))), labels = assets)
plt.show()